## F-distribution and F-test

### F-distribution:

F-distribution - right-skewed distribution used most commonly in ANOVA (Analysis of Variance). - ( https://en.wikipedia.org/wiki/F-distribution ) 
named after Ronald Fisher and George W. Snedecor

Ronald Fisher 1890-1962, British, Developed the analysis of variance ANOVA.
George W. Snedecor, 1881-1974, USA 

A random variate of the F-distribution

$ X = \frac{(\frac{U_1}{d_1})} {(\frac {U_2}{d_2})} $

where U1 and U2 are independent and have chi-squared distributions 
with d1 & d2 degrees of freedom respectively.

#### F-tests: 

The F-statistic is the ratio of the between-group variance to the within-group variance.

    var_between / var_within

If this test statistic is large, this suggests that the variation between groups is much higher than the variation within a group.

Suppose we have two "populations" of data. They both have normal distributions and are independent from each other.
<br>
<br>We can check two different Null Hypotheses:
<br>
<br>Test equality of variances (calculate statistics (var1/var2) and p-value)
<br>Test equality of means (also calculate statistics and p-value)
<br> --------------------------------------------------------------
<br>For testing variances, the F-test statistic is simply:
<br>
<br>    f_stat = Var(X) / Var(Y)
<br>
<br>and is distributed as df1 = len(X) - 1, df2 = len(Y) - 1
<br>
``` python
from scipy import stats
alpha = 0.05
p_val = 1 - stats.f.cdf(f_stat, df1, df2) 
# p_val = stats.f.sf(f_stat, df1, df2)
if p_value > alpha:
    print("Reject the null hypothesis that Var(X) == Var(Y)")
```
<br> --------------------------------------------------------------
<br>For testing means you can use:
<br>
``` python
import scipy.stats as stats
f_stat, p_val = stats.f_oneway(a,b)
```

A good tutorial can be found here: https://www.khanacademy.org/math/probability/statistics-inferential/anova/v/anova-1-calculating-sst-total-sum-of-squares
<br>
<br> --------------------------------------------------------------
Note that the F-test is extremely sensitive to non-normality of X and Y, so you're probably better off doing a more robust test such as Levene's test (or may be Bartlett's test) unless you're reasonably sure that X and Y are distributed normally.

Levene's test
http://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.levene.html

Bartlett's test
http://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.bartlett.html

See discussin here: https://stackoverflow.com/questions/21494141/how-do-i-do-a-f-test-in-python/21503346

Look further:
 - sklearn.feature_selection.f_classif - ANOVA tests
 - sklearn.feature_selection.f_regression - does sequential testing of regressions (compare betwen models for feaute selection)

In [1]:
import sys,os
import pandas as pd
import numpy as np

from scipy import stats
import math

import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'retina'

In [2]:
# test that two populations have same variance
# Null Hypothesis: var1 == var2

# ------------------------------------------
def f_test1(data1, data2, alpha=0.05):
    """
    """
    # import numpy as np
    # from scipy import stats 
    x1 = np.array(data1)
    x2 = np.array(data2)

    # ddof  = Delta Degrees of Freedom. 
    # For example, when you calculate, a standard deviation, 
    # the divisor used in calculations is N - ddof, 
    # where N represents the number of elements. 
    
    var_x1 = np.var(x1, ddof=1)
    var_x2 = np.var(x2, ddof=1)
    f_stat = var_x1 / var_x2

    dfn = x1.size-1 # degrees of freedom numerator 
    dfd = x2.size-1 # degrees of freedom denominator 
    
    # find p-value of F-test statistic
    p_val = 1 - stats.f.cdf(f_stat, dfn, dfd)  

    return f_stat, p_val

# ------------------------------------------
def f_test2(data1, data2, alpha=0.05):
    """
    """
    # import numpy as np
    # from scipy import stats 
    x1 = np.array(data1)
    x2 = np.array(data2)

    var_x1 = np.var(x1, ddof=1)
    var_x2 = np.var(x2, ddof=1)
    f_stat = var_x1 / var_x2

    dfn = x1.size-1 # degrees of freedom numerator 
    dfd = x2.size-1 # degrees of freedom denominator 

    # F-distribution object
    f_dist = stats.f(dfn, dfd) 

    # critical value
    f_crit_val = f_dist.ppf(1-alpha)

    # P-value
    p_val = 2*min(f_dist.cdf(f_crit_val), 1-f_dist.cdf(f_crit_val))
    
    return f_stat, p_val

# ------------------------------------------
def reject_or_not(p_val, alpha):
    if (p_val < alpha):
        print(f"Reject H0")
    else:
        print(f"Do Not Reject H0")

In [3]:
# Test that variances are equal
print("H0: var1 == var2") # Null Hypothesis

alpha = 0.05
data1 = [1, 2,  1, 2, 1, 2,  1, 2,  1, 2]
data2 = [1, 3, -1, 2, 1, 5, -1, 6, -1, 2]
print(f"data1 : {np.mean(data1):.4f} +/- {np.std(data1):.4f}")
print(f"data2 : {np.mean(data2):.4f} +/- {np.std(data2):.4f}")

print("-"*40)
f_stat, p_val = f_test1(data1, data2, alpha=0.05)
print(f"f_stat     = {f_stat:.4f} = var_a / var_b")
print(f"p_val      = {p_val:.4f}")
reject_or_not(p_val, alpha)

print("-"*40)
f_stat, p_val = f_test2(data1, data2, alpha=0.05)
print(f"f_stat     = {f_stat:.4f} = var_a / var_b")
print(f"p_val      = {p_val:.4f}")
reject_or_not(p_val, alpha)

print("-"*40)

H0: var1 == var2
data1 : 1.5000 +/- 0.5000
data2 : 1.7000 +/- 2.3259
----------------------------------------
f_stat     = 0.0462 = var_a / var_b
p_val      = 1.0000
Do Not Reject H0
----------------------------------------
f_stat     = 0.0462 = var_a / var_b
p_val      = 0.1000
Do Not Reject H0
----------------------------------------


In [4]:
# Test that variances are equal - different data
print("H0: var1 == var2") # Null Hypothesis

data1 = [18, 19, 22, 25, 27, 28, 41, 45, 51, 55]
data2 = [14, 15, 15, 17, 18, 22, 25, 25, 27, 34]

print(f"data1 : {np.mean(data1):.4f} +/- {np.std(data1):.4f}")
print(f"data2 : {np.mean(data2):.4f} +/- {np.std(data2):.4f}")

print("-"*40)
f_stat, p_val = f_test1(data1, data2, alpha=0.05)
print(f"f_stat     = {f_stat:.4f} = var_a / var_b")
print(f"p_val      = {p_val:.4f}")
reject_or_not(p_val, alpha)

print("-"*40)
f_stat, p_val = f_test2(data1, data2, alpha=0.05)
print(f"f_stat     = {f_stat:.4f} = var_a / var_b")
print(f"p_val      = {p_val:.4f}")
reject_or_not(p_val, alpha)
print("-"*40)

H0: var1 == var2
data1 : 33.1000 +/- 12.9727
data2 : 21.2000 +/- 6.1935
----------------------------------------
f_stat     = 4.3871 = var_a / var_b
p_val      = 0.0191
Reject H0
----------------------------------------
f_stat     = 4.3871 = var_a / var_b
p_val      = 0.1000
Do Not Reject H0
----------------------------------------


In [5]:
# Perform one-way ANOVA
print("H0: mean1 == mean2") # Null Hypothesis

data1 = [18, 19, 22, 25, 27, 28, 41, 45, 51, 55]
data2 = [14, 15, 15, 17, 18, 22, 25, 25, 27, 34]
print(f"data1 : {np.mean(data1):.4f} +/- {np.std(data1):.4f}")
print(f"data2 : {np.mean(data2):.4f} +/- {np.std(data2):.4f}")

print("-"*40)
f_stat, p_val = stats.f_oneway(data1, data2)
print(f"f_stat     = {f_stat:.4f} = var_a / var_b")
print(f"p_val      = {p_val:.4f}")
reject_or_not(p_val, alpha)

print("-"*40)

H0: mean1 == mean2
data1 : 33.1000 +/- 12.9727
data2 : 21.2000 +/- 6.1935
----------------------------------------
f_stat     = 6.1674 = var_a / var_b
p_val      = 0.0231
Reject H0
----------------------------------------
